In [1]:
import tkinter as tk
from tkinter import ttk, messagebox, filedialog
from pytube import YouTube
from threading import Thread
import os
import time
import re

def format_size(size_in_bytes):
    for unit in ['B', 'KB', 'MB', 'GB']:
        if size_in_bytes < 1024.0:
            return f"{size_in_bytes:.2f} {unit}"
        size_in_bytes /= 1024.0

def get_video_id(url):
    try:
        video_id_match = re.search(r'(?:v=|\/)([0-9A-Za-z_-]{11}).*', url)
        if video_id_match:
            return video_id_match.group(1)
        else:
            raise ValueError("Could not extract video ID from the URL.")
    except Exception as e:
        raise ValueError(f"Error extracting video ID: {e}")

def download_video(url, resolution, directory):
    try:
        video_id = get_video_id(url)
        yt = YouTube(f"https://www.youtube.com/watch?v={video_id}")
        streams = yt.streams.filter(progressive=True, file_extension='mp4', resolution=resolution)
        stream = streams.first()

        progress_window = tk.Toplevel(main_window)
        progress_window.title("Download Progress")

        status_label = tk.Label(progress_window, text="Downloading...", font=("Arial", 12))
        status_label.pack(pady=10)

        progress_bar = ttk.Progressbar(progress_window, orient="horizontal", length=300, mode="determinate", maximum=100)
        progress_bar.pack(pady=10)

        size_label = tk.Label(progress_window, text="", font=("Arial", 10))
        size_label.pack(pady=10)

        def update_progress():
            total_size = stream.filesize
            while True:
                bytes_downloaded = os.path.getsize(os.path.join(directory, stream.default_filename))
                percentage = (bytes_downloaded / total_size) * 100
                status_label.config(text=f"Downloading... {percentage:.2f}%")
                progress_bar["value"] = percentage
                size_label.config(text=f"Downloaded: {format_size(bytes_downloaded)} / {format_size(total_size)}")
                progress_window.update()
                if bytes_downloaded >= total_size:
                    break
                time.sleep(0.5)

            status_label.config(text="Download Complete!")
            progress_bar["value"] = 100
            show_completion_message(progress_window)

        Thread(target=update_progress).start()
        Thread(target=stream.download, args=(directory,)).start()
    except Exception as e:
        status_label.config(text=f"Failed to download video. Error: {e}")
        progress_bar["value"] = 0

def show_completion_message(window):
    messagebox.showinfo("Download Complete", "The download is complete!")
    window.destroy()
    main_window.destroy()

def close_program(window):
    window.destroy()
    main_window.destroy()

def choose_directory():
    directory = filedialog.askdirectory()
    if directory:
        directory_var.set(directory)

def check_resolutions():
    url = url_var.get()
    resolutions = get_available_resolutions(url)
    resolution_var.set(resolutions[0] if resolutions else "No available resolutions")
    resolution_menu["values"] = resolutions

def get_available_resolutions(video_url, file_extension='mp4'):
    try:
        video_id = get_video_id(video_url)
        yt = YouTube(f"https://www.youtube.com/watch?v={video_id}")
        return [stream.resolution for stream in yt.streams.filter(progressive=True, file_extension=file_extension)]
    except Exception as e:
        print(f"Error: {e}")
        return []

# Create the main window
main_window = tk.Tk()
main_window.title("YouTube Video Downloader")

# Create and place widgets in the window
tk.Label(main_window, text="YouTube Video URL:").grid(row=0, column=0, padx=10, pady=10)
url_var = tk.StringVar()
url_entry = tk.Entry(main_window, textvariable=url_var, width=40)
url_entry.grid(row=0, column=1, padx=10, pady=10)

# Dropdown for resolution
resolution_var = tk.StringVar()
resolution_var.set("No available resolutions")
resolutions = get_available_resolutions("")
resolution_menu = ttk.Combobox(main_window, textvariable=resolution_var, values=resolutions)
resolution_menu.grid(row=1, column=1, padx=10, pady=10)

# Check Resolutions button
check_resolutions_button = tk.Button(main_window, text="Check Resolutions", command=check_resolutions)
check_resolutions_button.grid(row=1, column=2, padx=10, pady=10)

# Entry for directory
directory_var = tk.StringVar()
directory_entry = tk.Entry(main_window, textvariable=directory_var, width=30)
directory_entry.grid(row=2, column=1, padx=10, pady=10)
choose_directory_button = tk.Button(main_window, text="Choose Directory", command=choose_directory)
choose_directory_button.grid(row=2, column=2, padx=10, pady=10)

# Download button
download_button = tk.Button(main_window, text="Download Video", command=lambda: download_video(url_var.get(), resolution_var.get(), directory_var.get()))
download_button.grid(row=3, column=0, columnspan=3, pady=20)

# Run the main loop
main_window.mainloop()


Error: Error extracting video ID: Could not extract video ID from the URL.
